# Producer

In [33]:
!pip3 install confluent_kafka

In [34]:
import json
import time
import logging
import uuid

from datetime import datetime
from urllib.request import urlopen
from confluent_kafka import Producer
from typing import Dict, List, Optional, Any
from datetime import datetime, timedelta
from urllib.parse import urlencode

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [35]:
session_key = '9161'
driver_number = '63'
lap_number = '2'

In [36]:
url = f'https://api.openf1.org/v1/laps?session_key={session_key}&driver_number={driver_number}&lap_number={lap_number}'

# Send request to the API
response = urlopen(url)

# Parse the JSON response
data = json.loads(response.read().decode('utf-8'))

# Extract the start time for Lap n (from the 'date_start' field)
if data:  # Check if data is not empty
    lap_start_time = data[0].get("date_start", None)
    
    if lap_start_time:
        print(f"Start time for Lap {lap_number}: {lap_start_time}")
    else:
        print(f"No start time found for Lap {lap_number}.")
else:
    print(f"No data found for Lap {lap_number}.")


Start time for Lap 2: 2023-09-16T13:05:40.397000+00:00


In [37]:

# Example lap start time
lap_start_time =lap_start_time 

# Convert lap_start_time to datetime object
start_dt = datetime.fromisoformat(lap_start_time)

# Calculate the end time (5 minutes after the start time)
end_dt = start_dt + timedelta(minutes=5)

# Format the start and end times in the required ISO 8601 format
start_time = start_dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')
end_time = end_dt.strftime('%Y-%m-%dT%H:%M:%S.000Z')


In [38]:
# First define our parameters
parameters = {
     
    'session':session_key,
    'driver': driver_number,
    'start_time': start_time ,
    'end_time': end_time,
    'data_types': ['car']
}

In [ ]:
class Args:
    def __init__(self, params):
        self.param1 = params['start_time']
        self.param2 = params['end_time']
        self.session = params['session']
        self.driver = params['driver']
        self.data_types = params['data_types']

args = Args(parameters)
class F1SingleStoreProducer:
    def __init__(self):
        self.producer = None
        self.session_key = args.session
        self.start_time = args.param1
        self.driver_number = args.driver
        self.end_time = args.param2
        self.base_url = "https://api.openf1.org/v1"

        self.kafka_config = {
            'bootstrap.servers': 'pkc-p11xm.us-east-1.aws.confluent.cloud:9092',
            'security.protocol': 'SASL_SSL',
            'sasl.mechanisms': 'PLAIN',
            'sasl.username': 'YWFLJGAW7TCIGMHG',
            'sasl.password': '5Fo+Oq9WbFhpKzF30goFhIVtZpzC9B9HkUHHgOWCQq721rCgDQGsefqH31cJzLEs',
            'client.id': 'f1-data-producer',
            'acks': 'all',
            'retries': 3,
            'retry.backoff.ms': 1000,
            'session.timeout.ms': 45000,
            'linger.ms': 0,
            'batch.size': 100,
            'compression.type': 'none',
            'max.in.flight.requests.per.connection': 1000,
            'queue.buffering.max.messages': 100,
            'queue.buffering.max.ms': 5
        }
        

        self.data_types = {
            'location': {
                'endpoint': 'location',
                'topic': 'topic_0',
                'schema': {
                    "type": "struct",
                    "fields": [
                        {"type": "string", "optional": False, "field": "id"},
                        {"type": "float", "optional": True, "field": "x"},
                        {"type": "float", "optional": True, "field": "y"},
                        {"type": "float", "optional": True, "field": "z"},
                        {"type": "int32", "optional": False, "field": "driver_number"},
                        {"type": "string", "optional": True, "field": "date"},
                        {"type": "int32", "optional": False, "field": "session_key"},
                        {"type": "int32", "optional": False, "field": "meeting_key"}
                    ],
                    "optional": False,
                    "name": "f1_location_data"
                }
            },
            'position': {
                'endpoint': 'position',
                'topic': 'topic_3',
                'schema': {
                    "type": "struct",
                    "fields": [
                        {"type": "string", "optional": False, "field": "id"},
                        {"type": "int32", "optional": True, "field": "position"},
                        {"type": "int32", "optional": False, "field": "driver_number"},
                        {"type": "string", "optional": True, "field": "date"},
                        {"type": "int32", "optional": False, "field": "session_key"},
                        {"type": "int32", "optional": False, "field": "meeting_key"}
                    ],
                    "optional": False,
                    "name": "f1_position_data"
                }
            },
            'car': {
                'endpoint': 'car_data',
                'topic': 'topic_1',
                'schema': {
                    "type": "struct",
                    "fields": [
                        {"type": "string", "optional": False, "field": "id"},
                        {"type": "int32", "optional": True, "field": "brake"},
                        {"type": "int32", "optional": True, "field": "drs"},
                        {"type": "int32", "optional": True, "field": "n_gear"},
                        {"type": "int32", "optional": True, "field": "rpm"},
                        {"type": "int32", "optional": True, "field": "speed"},
                        {"type": "int32", "optional": True, "field": "throttle"},
                        {"type": "int32", "optional": False, "field": "driver_number"},
                        {"type": "string", "optional": True, "field": "date"},
                        {"type": "int32", "optional": False, "field": "session_key"},
                        {"type": "int32", "optional": False, "field": "meeting_key"}
                    ],
                    "optional": False,
                    "name": "f1_car_data"
                }
            },
            'intervals': {
                'endpoint': 'intervals',
                'topic': 'topic_2',
                'schema': {
                    "type": "struct",
                    "fields": [
                        {"type": "string", "optional": False, "field": "id"},
                        {"type": "int32", "optional": False, "field": "driver_number"},
                        {"type": "float", "optional": True, "field": "gap_to_leader"},
                        {"type": "float", "optional": True, "field": "time_interval"},
                        {"type": "string", "optional": True, "field": "date"},
                        {"type": "int32", "optional": False, "field": "session_key"},
                        {"type": "int32", "optional": False, "field": "meeting_key"}
                    ],
                    "optional": False,
                    "name": "f1_intervals_data"
                }
            }
        }

    def delivery_callback(self, err, msg):
        if err is not None:
            logger.error(f'Message delivery failed: {err}')
        else:
            logger.info(f'Message delivered to {msg.topic()} [{msg.partition()}] @ {msg.offset()}')

    def connect_kafka(self) -> bool:
        try:
            if self.producer is not None:
                self.producer.close()
                self.producer = None

            logger.info("Connecting to Confluent Cloud Kafka...")
            self.producer = Producer(self.kafka_config)
            logger.info("Successfully connected to Confluent Cloud")
            return True

        except Exception as e:
            logger.error(f"Failed to connect to Kafka: {e}")
            return False

    def get_f1_data(self, data_type: str):
        endpoint = self.data_types[data_type]['endpoint']
        url = (f"{self.base_url}/{endpoint}"
               f"?session_key={self.session_key}"
               f"&driver_number={self.driver_number}"
               f"&date>{self.start_time}"
               f"&date<{self.end_time}")

        try:
            logger.info(f"Fetching {data_type} data from: {url}")
            response = urlopen(url)
            data = json.loads(response.read().decode('utf-8'))
            logger.info(f"Retrieved {len(data)} records")
            return data
        except Exception as e:
            logger.error(f"Error fetching F1 data: {e}")
            return []

    def prepare_record(self, record: dict, data_type: str) -> dict:
        try:
            formatted_record = self.format_record(record, data_type)
            return {
                "schema": self.data_types[data_type]['schema'],
                "payload": formatted_record
            }
        except Exception as e:
            logger.error(f"Error preparing record: {e}")
            raise

    def format_record(self, record: dict, data_type: str) -> dict:
        formatted_record = {"id": str(uuid.uuid4())}

        if data_type == 'location':
            formatted_record.update({
                "x": float(record.get('x', 0.0)),
                "y": float(record.get('y', 0.0)),
                "z": float(record.get('z', 0.0))
            })
        elif data_type == 'position':
            formatted_record.update({
                "position": int(record.get('position', 0))
            })
        elif data_type == 'intervals':
            formatted_record.update({
                "gap_to_leader": float(record.get('gap_to_leader', 0.0)) if record.get('gap_to_leader') not in [None, '+1 LAP'] else None,
                "time_interval": float(record.get('interval', 0.0)) if record.get('interval') not in [None, '+1 LAP'] else None
            })
        elif data_type == 'car':
            formatted_record.update({
                "brake": int(record.get('brake', 0)),
                "drs": int(record.get('drs', 0)),
                "n_gear": int(record.get('n_gear', 0)),
                "rpm": int(record.get('rpm', 0)),
                "speed": int(record.get('speed', 0)),
                "throttle": int(record.get('throttle', 0))
            })

        date_str = record.get('date', '')
        formatted_record.update({
            "driver_number": int(record.get('driver_number', 0)),
            "date": datetime.fromisoformat(date_str.replace('+00:00', '')).strftime('%Y-%m-%d %H:%M:%S') if date_str else None,
            "session_key": int(record.get('session_key', 0)),
            "meeting_key": int(record.get('meeting_key', 0))
        })

        return formatted_record

    def prepare_key(self, driver_number: int) -> dict:
        return {
            "schema": {
                "type": "struct",
                "fields": [
                    {"type": "int32", "optional": False, "field": "driver_number"}
                ],
                "optional": False,
                "name": "f1_driver_key"
            },
            "payload": {
                "driver_number": int(driver_number)
            }
        }

    def send_record(self, data_type: str, record: dict, record_num: int, total: int) -> bool:
        try:
            if self.producer is None and not self.connect_kafka():
                return False

            prepared_record = self.prepare_record(record, data_type)
            driver_number = record.get('driver_number', 0)
            prepared_key = self.prepare_key(driver_number)
            topic = self.data_types[data_type]['topic']

            logger.info(f"Sending {data_type} record {record_num}/{total} for driver {driver_number}")

            key_bytes = json.dumps(prepared_key).encode('utf-8')
            value_bytes = json.dumps(prepared_record).encode('utf-8')

            self.producer.produce(
                topic=topic,
                key=key_bytes,
                value=value_bytes,
                callback=self.delivery_callback
            )

            self.producer.poll(0)

            return True

        except Exception as e:
            logger.error(f"Error sending record: {e}")
            if self.producer is not None:
                self.producer.close()
                self.producer = None
            return False

    def process_data(self, data_type: str, data: list) -> None:
        if not data:
            logger.warning(f"No {data_type} data to process")
            return

        for i, record in enumerate(data, 1):
            success = False
            retries = 0
            while not success and retries < 3:
                success = self.send_record(data_type, record, i, len(data))
                if not success:
                    retries += 1
                    time.sleep(1)
            if not success:
                logger.error(f"Failed to send {data_type} record after {retries} attempts")

            self.producer.flush(timeout=10)
            time.sleep(0.1)

    def run(self, data_types=None):
        if data_types is None:
            data_types = list(self.data_types.keys())
        elif isinstance(data_types, str):
            data_types = [data_types]

        logger.info(f"Starting F1 data producer for: {', '.join(data_types)}")

        try:
            for data_type in data_types:
                if data_type not in self.data_types:
                    logger.error(f"Unknown data type: {data_type}")
                    continue

                logger.info(f"\n=== Processing {data_type.title()} Data ===")
                data = self.get_f1_data(data_type)
                self.process_data(data_type, data)

        except KeyboardInterrupt:
            logger.info("\nShutting down...")
        finally:
            if self.producer:
                self.producer.flush()
                #self.producer.close()
                logger.info("Producer closed")

# Now run the producer
try:
    producer = F1SingleStoreProducer()
    producer.run(args.data_types)
except Exception as e:
    print(f"Fatal error: {e}")

INFO:__main__:Starting F1 data producer for: car
INFO:__main__:
=== Processing Car Data ===
INFO:__main__:Fetching car data from: https://api.openf1.org/v1/car_data?session_key=9161&driver_number=63&date>2023-09-16T13:05:40.000Z&date<2023-09-16T13:10:40.000Z
INFO:__main__:Retrieved 1137 records
INFO:__main__:Connecting to Confluent Cloud Kafka...
%4|1740496122.678|CONFWARN|f1-data-producer#producer-1| [thrd:app]: Configuration property session.timeout.ms is a consumer property and will be ignored by this producer instance
INFO:__main__:Successfully connected to Confluent Cloud
INFO:__main__:Sending car record 1/1137 for driver 63
%6|1740496122.834|GETSUBSCRIPTIONS|f1-data-producer#producer-1| [thrd:main]: Telemetry client instance id changed from AAAAAAAAAAAAAAAAAAAAAA to whAxHA4cSd2dm1oOy8S/mw
INFO:__main__:Message delivered to topic_1 [2] @ 31234
INFO:__main__:Sending car record 2/1137 for driver 63
INFO:__main__:Message delivered to topic_1 [2] @ 31235
INFO:__main__:Sending car reco